In [1]:
!pip install --upgrade lightgbm

  Obtaining dependency information for lightgbm from https://files.pythonhosted.org/packages/d9/28/3be76b591a2e14a031b681b8283acf1dec2ad521f6f1701b7957df68c466/lightgbm-4.5.0-py3-none-win_amd64.whl.metadata
   ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
    --------------------------------------- 0.0/1.4 MB 640.0 kB/s eta 0:00:03
   -------------------- ------------------- 0.7/1.4 MB 9.2 MB/s eta 0:00:01
   ---------------------------------------- 1.4/1.4 MB 13.0 MB/s eta 0:00:00


In [2]:
import lightgbm as lgm
import pandas as pd

In [3]:
dataset=pd.read_csv("insurance_pre.csv")
dataset

,age,sex,bmi,children,smoker,charges
0,19,female,27.900,0,yes,16884.92400
1,18,male,33.770,1,no,1725.55230
2,28,male,33.000,3,no,4449.46200
3,33,male,22.705,0,no,21984.47061
4,32,male,28.880,0,no,3866.85520
...,...,...,...,...,...,...
1333,50,male,30.970,3,no,10600.54830
1334,18,female,31.920,0,no,2205.98080
1335,18,female,36.850,0,no,1629.83350
1336,21,female,25.800,0,no,2007.94500


In [4]:
dataset=pd.get_dummies(dataset,drop_first=True)
dataset

,age,bmi,children,charges,sex_male,smoker_yes
0,19,27.900,0,16884.92400,False,True
1,18,33.770,1,1725.55230,True,False
2,28,33.000,3,4449.46200,True,False
3,33,22.705,0,21984.47061,True,False
4,32,28.880,0,3866.85520,True,False
...,...,...,...,...,...,...
1333,50,30.970,3,10600.54830,True,False
1334,18,31.920,0,2205.98080,False,False
1335,18,36.850,0,1629.83350,False,False
1336,21,25.800,0,2007.94500,False,False


In [5]:
dataset.columns

Index(['age', 'bmi', 'children', 'charges', 'sex_male', 'smoker_yes'], dtype='object')

In [6]:
independent=dataset[['age', 'bmi', 'children','sex_male', 'smoker_yes']]
independent

,age,bmi,children,sex_male,smoker_yes
0,19,27.900,0,False,True
1,18,33.770,1,True,False
2,28,33.000,3,True,False
3,33,22.705,0,True,False
4,32,28.880,0,True,False
...,...,...,...,...,...
1333,50,30.970,3,True,False
1334,18,31.920,0,False,False
1335,18,36.850,0,False,False
1336,21,25.800,0,False,False


In [7]:
dependent=dataset[['charges']]
dependent

,charges
0,16884.92400
1,1725.55230
2,4449.46200
3,21984.47061
4,3866.85520
...,...
1333,10600.54830
1334,2205.98080
1335,1629.83350
1336,2007.94500


In [8]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(independent,dependent,test_size=0.30,random_state=0)

In [10]:
x_train

,age,bmi,children,sex_male,smoker_yes
1163,18,28.215,0,False,False
196,39,32.800,0,False,False
438,52,46.750,5,False,False
183,44,26.410,0,False,False
1298,33,27.455,2,True,False
...,...,...,...,...,...
763,27,26.030,0,True,False
835,42,35.970,2,True,False
1216,40,25.080,0,True,False
559,19,35.530,0,True,False


In [11]:
x_test

,age,bmi,children,sex_male,smoker_yes
578,52,30.200,1,True,False
610,47,29.370,1,False,False
569,48,40.565,2,True,True
1034,61,38.380,0,True,False
198,51,18.050,0,False,False
...,...,...,...,...,...
1261,28,37.100,1,True,False
494,21,25.700,4,True,True
97,55,38.280,0,True,False
418,64,39.160,1,True,False


In [12]:
from lightgbm import LGBMRegressor
regressor=LGBMRegressor(num_threads=2,learning_rate=0.05,num_iterations=100,max_depth=100,colsample_bytree=0.8,reg_alpha=1e-2,reg_lambda=1e-2,seed=42)
regressor.fit(x_train,y_train.values.ravel())

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000051 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 313
[LightGBM] [Info] Number of data points in the train set: 936, number of used features: 5
[LightGBM] [Info] Start training from score 13232.916456


C:\Users\Malya\anaconda3\Lib\site-packages\lightgbm\engine.py:204: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


LGBMRegressor(colsample_bytree=0.8, learning_rate=0.05, max_depth=100,
              num_iterations=100, num_threads=2, reg_alpha=0.01,
              reg_lambda=0.01, seed=42)

In [13]:
y_pred=regressor.predict(x_test)

In [14]:
from sklearn.metrics import r2_score
r_score=r2_score(y_test,y_pred)
r_score

0.87823584126943

In [15]:
import pickle
filename="finalize_LGBoost_sav"
pickle.dump(regressor,open(filename,"wb"))

In [16]:
loaded_model=pickle.load(open("finalize_LGBoost_sav","rb"))
result=loaded_model.predict([[19,27.9,0,1,0]])

In [17]:
print(result)

[2969.78961102]
